In [13]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep
pd.options.mode.chained_assignment = None  # default='warn'

In [14]:
# API Keys
import config
print(config.api_seeds_key)

iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH


# Load Top 10 Songs

In [38]:
ENRICHED_WKLY_FILE_NAME = os.path.join('data', 'clean', 'regional-us-weekly-enriched.csv')
ENRICHED_WORST_FILE_NAME = os.path.join('data', 'clean', '2018-worst-songs-enriched.csv')
ENRICHED_FORGOTIFY_FILE_NAME = os.path.join('data', 'clean', 'forgotify-10-enriched.csv')
LYRICS_DIR = os.path.join('data', 'lyrics')

In [16]:
wkly_data_df = pd.read_csv(ENRICHED_WKLY_FILE_NAME)
track_and_features_df = wkly_data_df.drop(['Position', 'URL', 'Week Start Date', 'Week End Date', 'Album'], axis=1)
agg_by_track_df = track_and_features_df.groupby(['Track Name', 'Artist']).agg({
    'Streams':'sum', 'Acousticness':'max', 'Danceability':'max', 'Key':'min', 'Liveness':'max', 'Loudness':'max',
    'Popularity':'max', 'Speechiness':'max', 'Tempo':'max', 'Time Signature':'max', 'Valence':'max', 'Energy':'max', 'Instrumentalness':'max', 'Mode':'max'}) 
sorted_by_total_streams_df = agg_by_track_df.sort_values(by="Streams", ascending=False).reset_index()
top_10_df = sorted_by_total_streams_df.head(n=10)
top_10_df['Label'] = top_10_df['Track Name'] + ' (' + top_10_df['Artist'] + ")"
top_10_df

,Track Name,Artist,Streams,Acousticness,Danceability,Key,Liveness,Loudness,Popularity,Speechiness,Tempo,Time Signature,Valence,Energy,Instrumentalness,Mode,Label
0,God's Plan,Drake,460070262,0.03320,0.758,7,0.558,-9.211,90,0.1090,77.179,4,0.373,0.449,0.000087,1,God's Plan (Drake)
1,SAD!,XXXTENTACION,340496175,0.25800,0.740,8,0.123,-4.880,92,0.1450,75.023,4,0.473,0.613,0.003720,1,SAD! (XXXTENTACION)
2,Psycho (feat. Ty Dolla $ign),Post Malone,310156476,0.59000,0.748,8,0.112,-8.011,88,0.1170,140.124,4,0.439,0.559,0.000000,1,Psycho (feat. Ty Dolla $ign) (Post Malone)
3,Lucid Dreams,Juice WRLD,303942353,0.37700,0.511,6,0.340,-7.230,89,0.2370,83.903,4,0.218,0.566,0.000000,0,Lucid Dreams (Juice WRLD)
4,Look Alive (feat. Drake),BlocBoy JB,268178220,0.00104,0.922,10,0.105,-7.495,86,0.2700,140.022,4,0.595,0.581,0.000059,1,Look Alive (feat. Drake) (BlocBoy JB)
5,Nice For What,Drake,267229876,0.09340,0.700,8,0.119,-6.461,86,0.0736,93.469,4,0.792,0.913,0.000124,1,Nice For What (Drake)
6,Better Now,Post Malone,254685346,0.35400,0.680,10,0.136,-5.843,92,0.0454,145.028,4,0.374,0.563,0.000000,1,Better Now (Post Malone)
7,Moonlight,XXXTENTACION,249897971,0.55600,0.921,9,0.102,-5.723,91,0.0804,128.009,4,0.711,0.537,0.004040,0,Moonlight (XXXTENTACION)
8,In My Feelings,Drake,247009164,0.05890,0.835,1,0.448,-5.833,90,0.1390,181.995,4,0.350,0.626,0.014400,1,In My Feelings (Drake)
9,I Like It,Cardi B,239838666,0.09900,0.816,5,0.372,-3.998,90,0.1290,136.048,4,0.650,0.726,0.000000,0,I Like It (Cardi B)


In [44]:
for ii in range(0,10):
    row = top_10_df.iloc[ii, :]
    track_name = row['Track Name']
    artist = row['Artist']
    label = row['Label']
    url = f'https://orion.apiseeds.com/api/music/lyric/{artist}/{track_name}?apikey={config.api_seeds_key}'
    print(url)
    lyrics_rsp = requests.get(url)
    lyrics_json = lyrics_rsp.json()
    if 'error' in lyrics_json:
        print(f'lyrics not found skipping')
        continue
    filename = os.path.join(LYRICS_DIR, label)
    with open(filename, 'w+') as fp:
        fp.write(lyrics_json['result']['track']['text'])

https://orion.apiseeds.com/api/music/lyric/Drake/God's Plan?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
https://orion.apiseeds.com/api/music/lyric/XXXTENTACION/SAD!?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
lyrics not found skipping
https://orion.apiseeds.com/api/music/lyric/Post Malone/Psycho (feat. Ty Dolla $ign)?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
https://orion.apiseeds.com/api/music/lyric/Juice WRLD/Lucid Dreams?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
https://orion.apiseeds.com/api/music/lyric/BlocBoy JB/Look Alive (feat. Drake)?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
https://orion.apiseeds.com/api/music/lyric/Drake/Nice For What?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6WrPTDnhVR5c5kPU8h9SyFGbHxSn7HdHzGxH
lyrics not found skipping
https://orion.apiseeds.com/api/music/lyric/Post Malone/Better Now?apikey=iSDWf29EfxSZCNhEc512k9r9z2OL6